In [ ]:
import os
from dotenv import load_dotenv
import psycopg2
from psycopg2 import sql
from psycopg2.extras import Json
from pgvector.psycopg2 import register_vector
from openai import OpenAI
from pypdf import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter, SentenceTransformersTokenTextSplitter
from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader
import hashlib
import uuid
import time
import ollama

# UTILITY FUNCTIONS

In [ ]:
# function to directly execute a query
def postgres_execute(conn, query, get_result=False, commit=True, print_query=False):
    if print_query: 
        print(query)
    with conn.cursor(cursor_factory=psycopg2.extras.RealDictCursor) as cursor:
    #with conn.cursor() as cursor:
        cursor.execute(query)
        if commit:
            conn.commit()
        if get_result:
            return cursor.fetchall()
        else:
            return None

In [ ]:
# Enhanced get_ollama_embedding function with model parameter
def get_ollama_embedding(text, myModel="mistral"):
    response = ollama.embeddings(model=myModel, prompt=text)
    embedding = response["embedding"]
    token_used = -1 # Ollama doesn't return the token used
    result = {}
    result["embedding"] = embedding
    result["token"] = token_used
    return result

In [ ]:
# Function to query similar documents with enhanced model selection
def query_documents(embedding_method, question, cur, conn, query_ledger_uuid,n_results=3, myModel="mistral", document_uuid=None,username='system'):
    """Query documents for relevant chunks based on embedding similarity.
    
    Args:
        embedding_method (str): Method for embeddings ('OLLAMA' or 'OPENAI')
        clientopenAI: OpenAI client (required if embedding_method is 'OPENAI')
        question (str): The question to search for
        cur: Database cursor
        conn: Database connection
        n_results (int): Number of top chunks to return (default: 3)
        myModel (str): Model name for OLLAMA embeddings (default: 'mistral')
        document_uuid (str, optional): If provided, search only within this document
        
    Returns:
        list: List of chunks ordered by relevance (vector similarity)
    """
    # ----------------------------------------------------------------------------------
    # -------------------------- WE EXTRACT THE SUB SET OF CHUNCK TO EMBED -------------
    start_time = time.time()
    targetChunckTable = ""
    embedding_method == "OLLAMA"
    # Using the enhanced table for generic embedding models
    targetChunckTable = "document_embeding_mistral_generic"
    query_embedding = get_ollama_embedding(question, myModel)['embedding']

    end_time = time.time()
    print("Time to get the embedding: ", end_time - start_time)
    print(f"Query: {question}")
    
    start_time = time.time()
    
    # Base query with all fields
    query = """SELECT 
        d.document_uuid as document_uuid,
        d.document_name as document_name,
        d.document_location as document_location,
        d.document_hash as document_hash,
        d.document_type as document_type,
        d.document_status as document_status,
        d.tags as document_tags,
        dc.document_chunk_uuid as document_chunk_uuid,
        dc.embebed_text as embebed_text,
        emb.document_embeding_uuid as document_embeding_uuid,
        emb.embeder_type as embeder_type,
        emb.embedding_token as embedding_token,
        emb.embedding_time as embedding_time,
        embedding <-> %s::vector as similarity_score
    """
    params = [query_embedding]
    
    query += """
    FROM document_library.""" + targetChunckTable + """ emb
    INNER JOIN document_library.document_chunks dc ON dc.document_chunk_uuid = emb.document_chunk_uuid 
    INNER JOIN document_library.documents d ON d.document_uuid = dc.document_uuid
    inner join document_library.document_security_groups dsg on dsg.document_uuid = d.document_uuid
    inner join document_library.security_groups sg on sg.security_group_uuid  = dsg.security_group_uuid
    inner join document_library.user_security_groups usg on usg.security_group_uuid = sg.security_group_uuid
    inner join document_library.users u on u.user_uuid = usg.user_uuid and u.sso_unique_id = %s
    """
    params.append(username)    
    # Add document filter if UUID is provided
    if document_uuid:
        query += "\nWHERE d.document_uuid = %s"
        params.append(document_uuid)

    query += "\nORDER BY similarity_score ASC"
    
    # Add limit
    query += "\nLIMIT %s"
    params.append(n_results)
    
    # Execute query
    query = cur.mogrify(query, params).decode('utf-8')
    #print(query)  # Debug output
    results = postgres_execute(conn, query, commit=True, get_result=True, print_query=True)
    end_time = time.time()
    print("Time to get the chunks from the db: ", end_time - start_time)
    print(f"Retrieved {len(results)} chunks" + (f" for document {document_uuid}" if document_uuid else ""))
    
    # If similarity scores were calculated, print them for debugging
    if results and 'similarity_score' in results[0]:
        print("\nTop chunks by similarity:")
        for i, chunk in enumerate(results[:min(5, len(results))]):
            score = chunk['similarity_score']
            text_preview = chunk['embebed_text'][:50] + "..." if len(chunk['embebed_text']) > 50 else chunk['embebed_text']
            print(f"{i+1}. Score: {score:.6f} - Document: {chunk['document_name']} - Text: {text_preview}")
            query_ledger_chunkk = f"""
            INSERT INTO document_library.query_answer_chunks
            (query_answer_chunk_uuid, query_answer_uuid, query_ledger_uuid, chunk_uuid, creation_date, created_by, updated_date, updated_by, "comments")    
            VALUES ('{str(uuid.uuid4())}', 'to_be_managed_if_needed', '{query_ledger_uuid}', '{chunk['document_chunk_uuid']}', now(), 'system', now(), 'system', 'chunk used for the answer')
            """
            postgres_execute(conn, query_ledger_chunkk, commit=True)
    elif results and document_uuid:
        print("\nTop chunks for this document:")
        for i, chunk in enumerate(results[:min(5, len(results))]):
            text_preview = chunk['embebed_text'][:50] + "..." if len(chunk['embebed_text']) > 50 else chunk['embebed_text']
            print(f"{i+1}. Document: {chunk['document_name']} - Text: {text_preview}")
            # we insert in the query ledger the chunk that was used
            # the structure of the table is INSERT INTO document_library.query_answer_chunks
            #(query_answer_chunk_uuid, query_answer_uuid, query_ledger_uuid, chunk_uuid, creation_date, created_by, updated_date, updated_by, "comments")    
            query_ledger_chunkk = f"""
            INSERT INTO document_library.query_answer_chunks
            (query_answer_chunk_uuid, query_answer_uuid, query_ledger_uuid, chunk_uuid, creation_date, created_by, updated_date, updated_by, "comments")    
            VALUES ('{str(uuid.uuid4())}', 'to_be_managed_if_needed', '{query_ledger_uuid}', '{chunk['document_chunk_uuid']}', now(), 'system', now(), 'system', 'chunk used for the answer')
            """
            postgres_execute(conn, query_ledger_chunkk, commit=True)
    
    return results

In [ ]:
# Function to generate response
def generate_response(answer_method, clientopenAI, question, relevant_chunks):
    context = "\n\n".join(relevant_chunks)
    prompt = (
        "You are an assistant for question-answering tasks. Use the retrieved context below "
        "to answer the question. If you don't know, say so in three sentences maximum."
        "\n\nContext:\n" + context + "\n\nQuestion:\n" + question
    )


    answer_method == "OPENAI"
    response = clientopenAI.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": prompt},
            {"role": "user", "content": question},
        ],
    )
    return response.choices[0].message.content

# METADATA FUNCTIONS

In [ ]:
# Function to retrieve metadata fields from the database
def get_metadata_fields(conn,cur,target_metadata_uuid=None):
    """Retrieve metadata fields and their configurations from the database.
    
    Args:
        conn: Database connection
        
    Returns:
        dict: Dictionary of metadata fields with their configurations
    """
    query = """
        SELECT 
            metadata_uuid,
            metadata_name,
            metadata_description,
            metadata_type
        FROM document_library.metadatas
    """
    if target_metadata_uuid:
        query += " WHERE metadata_uuid = %s"
        query = cur.mogrify(query, (target_metadata_uuid,)).decode('utf-8')
    with conn.cursor(cursor_factory=psycopg2.extras.RealDictCursor) as cursor:
        cursor.execute(query)
        metadata_records = cursor.fetchall()
    
    # Convert database records to the format needed for extraction
    metadata_fields = {}
    for record in metadata_records:
        # Create a prompt based on the metadata description or use a default prompt
        prompt_template = "You are an expert document analyzer. Extract the {field_description} from this text. Return ONLY the {output_format} or 'Not found' if it cannot be determined. Text: {{text}}"
        
        # Map database metadata_type to the appropriate value_type
        value_type_mapping = {
            "STRING": "string",
            "NUMBER": "float",
            "INTEGER": "int",
            "DATE": "date",
            "BOOLEAN": "boolean"
        }
        
        # Determine appropriate output format based on metadata type
        output_format = "value"
        if record["metadata_type"] == "DATE":
            output_format = "date in DD/MM/YYYY format"
        elif record["metadata_type"] == "NUMBER":
            output_format = "number (no currency symbol)"
        elif record["metadata_type"] == "INTEGER":
            output_format = "number as an integer"
        elif record["metadata_type"] == "BOOLEAN":
            output_format = "'true' or 'false'"
        
        value_type = value_type_mapping.get(record["metadata_type"].upper(), "string")
        
        metadata_fields[record["metadata_uuid"]] = {
            "name": record["metadata_name"],
            "prompt": prompt_template.format(
                field_description=record["metadata_description"], 
                output_format=output_format
            ),
            "confidence_threshold": 0.7,  # Default confidence threshold
            "value_type": value_type
        }
    
    return metadata_fields

In [ ]:
def extract_metadata_BEN2(document_uuid, conn, cur,client_llm=None, llm_method="OPENAI", overwrite_existing=False, embedding_method="OLLAMA", model_name="mistral", n_results_per_document=3,target_metadata_uuid=None, username="bfoucque"):

    # Get metadata fields from database
    metadata_fields = get_metadata_fields(conn,cur,target_metadata_uuid)
    
    print(f"Starting metadata extraction process for document: {document_uuid}")
    print(f"Processing {len(metadata_fields)} metadata fields")
    start_time = time.time()
    
    results = {}
    # Process each metadata field from the database
    for metadata_uuid, field_config in metadata_fields.items():            
        query_ledger_uuid = str(uuid.uuid4())
        # we insert in the query ledger the metadata that was used
        # the structure is INSERT INTO document_library.query_ledgers_extended
        #(query_ledger_uuid, query_type, query_content, metadata_uuid, user_uuid, query_tags, query_start_document_date, query_end_document_date, query_answer, creation_date, created_by, updated_date, updated_by, "comments")
        query_ledger = f"""
        INSERT INTO document_library.query_ledgers_extended
        (query_ledger_uuid, query_type, query_content, metadata_uuid, user_uuid, query_tags, query_start_document_date, query_end_document_date, query_answer, creation_date, created_by, updated_date, updated_by, "comments")
        VALUES ('{query_ledger_uuid}', 'metadata_extraction', '{field_config["prompt"].replace("'","''")}', '{metadata_uuid}', NULL, 'metadata_extraction', now(), now(), 'to_be_managed_if_needed', now(), 'system', now(), 'system', 'metadata extraction')
        """
        postgres_execute(conn, query_ledger, commit=True)

        print(f"Processing metadata field: {field_config['name']} (UUID: {metadata_uuid})")
        field_start_time = time.time()

        # Get all document chunks - modified to follow the flow in metadata_extraction.md
        print(f"Retrieving chunks for document: {document_uuid}")
        
        chunks = query_documents(
                embedding_method,
                field_config['prompt'],
                cur,
                conn,
                query_ledger_uuid=query_ledger_uuid,
                n_results=n_results_per_document,
                myModel=model_name,
                document_uuid=document_uuid,
                username=username
            )
        
        
        if not chunks:
            print(f"No chunks found for document UUID: {document_uuid}")
            
        # Concatenate all chunks text with section markers
        all_text = "\n\n---SECTION---\n\n".join([chunk["embebed_text"] for chunk in chunks])
        print(f"Retrieved {len(chunks)} chunks with total length: {len(all_text)} characters")
        
        # Format the prompt with the document text - step 3 in the flow
        prompt = field_config["prompt"].format(text=all_text)
        
        # Use the appropriate LLM to extract metadata - step 4 in the flow
        if llm_method == "OPENAI":
            if not client_llm:
                raise ValueError("OpenAI client required for OPENAI method")
                
            response = client_llm.chat.completions.create(
                model="gpt-4o-mini",  # Can be adjusted based on needs
                messages=[
                    {"role": "system", "content": prompt},
                ],
                temperature=0.1  # Low temperature for more deterministic results
            )
            raw_value = response.choices[0].message.content.strip()
            confidence = 0.9  
            model_used = "gpt-4o-mini"
            
        elif llm_method == "OLLAMA":
            # Using the specific model name parameter
            response = ollama.chat(
                model=model_name,
                messages=[{"role": "system", "content": prompt}],
            )
            raw_value = response['message']['content'].strip()
            confidence = 0.85
            model_used = model_name
        else:
            raise ValueError(f"Unsupported LLM method: {llm_method}")
        
        # Process the value based on the expected type
        value_type = field_config["value_type"]
        processed_value = None
        
        try:
            if value_type == "float":
                if raw_value.lower() == 'not found' or raw_value == '-1':
                    processed_value = None
                else:
                    numeric_str = ''.join(c for c in raw_value if c.isdigit() or c in '.-')
                    processed_value = float(numeric_str) if numeric_str else None
            
            elif value_type == "int":
                if raw_value.lower() == 'not found' or raw_value == '-1':
                    processed_value = None
                else:
                    numeric_str = ''.join(c for c in raw_value if c.isdigit() or c == '-')
                    processed_value = int(numeric_str) if numeric_str else None
            
            elif value_type == "date":
                if raw_value.lower() == 'not found':
                    processed_value = None
                else:
                    import datetime
                    from dateutil import parser
                    try:
                        # First try DD/MM/YYYY format
                        if '/' in raw_value:
                            day, month, year = map(int, raw_value.split('/'))
                            processed_value = datetime.date(year, month, day)
                        else:
                            processed_value = parser.parse(raw_value).date()
                    except (ValueError, parser.ParserError):
                        processed_value = None
            
            elif value_type == "boolean":
                processed_value = raw_value.lower() == 'true'
            
            elif value_type == "string":
                processed_value = raw_value if raw_value.lower() != 'not found' else None
                
        except Exception as e:
            print(f"Error processing value for {field_config['name']}: {e}")
            processed_value = None
        
        # Record the result with both raw and processed values
        results[metadata_uuid] = {
            "name": field_config['name'],
            "raw_value": raw_value,
            "processed_value": processed_value,
            "value_type": value_type,
            "confidence": confidence,
            "processing_time": time.time() - field_start_time
        }
        
        # MODIFIED SECTION: Check if metadata already exists for this document and field
        check_query = f"""
            SELECT document_metadata_uuid 
            FROM document_library.document_metadatas 
            WHERE document_uuid = '{document_uuid}' 
            AND metadata_uuid = '{metadata_uuid}'
        """
        existing_metadata = postgres_execute(conn, check_query, get_result=True)
        
        # Build metadata values dictionary with the processed value
        metadata_values = {
            "document_uuid": document_uuid,
            "metadata_uuid": metadata_uuid,
            "updated_date": "now()",
            "updated_by": "SYSTEM",
            "query_ledger_uuid": query_ledger_uuid,
            "comments": f"Extracted by {llm_method} model: {model_used}. Confidence: {confidence:.2f}"
        }
        
        # Set the appropriate value column based on type
        if value_type == "float" and processed_value is not None:
            metadata_values["metadata_value_float"] = processed_value
        elif value_type == "string" and processed_value is not None:
            metadata_values["metadata_value_string"] = processed_value
        elif value_type == "int" and processed_value is not None:
            metadata_values["metadata_value_int"] = processed_value
        elif value_type == "date" and processed_value is not None:
            metadata_values["metadata_value_date"] = processed_value
        elif value_type == "boolean":
            metadata_values["metadata_value_boolean"] = processed_value
            
        try:
            if existing_metadata and (overwrite_existing or len(existing_metadata) > 0):
                # Update existing record
                document_metadata_uuid = existing_metadata[0]['document_metadata_uuid']
                set_clauses = []
                params = []
                
                for key, value in metadata_values.items():
                    if key not in ["document_uuid", "metadata_uuid"]:  # Skip primary key fields
                        set_clauses.append(f"{key} = %s")
                        params.append(value)
                
                # Add WHERE clause
                params.append(document_metadata_uuid)
                
                update_query = f"""
                    UPDATE document_library.document_metadatas 
                    SET {", ".join(set_clauses)}
                    WHERE document_metadata_uuid = %s
                """
                
                with conn.cursor() as cursor:
                    cursor.execute(update_query, params)
                conn.commit()
                print(f"  - Updated {field_config['name']}: {raw_value[:50]}{'...' if len(raw_value) > 50 else ''}")
                
            else:
                # Insert new record
                document_metadata_uuid = str(uuid.uuid4())
                metadata_values["document_metadata_uuid"] = document_metadata_uuid
                metadata_values["creation_date"] = "now()"
                metadata_values["created_by"] = "SYSTEM"
                
                # Create SQL query for inserting metadata
                query = sql.SQL("INSERT INTO document_library.document_metadatas ({}) VALUES ({})").format(
                    sql.SQL(', ').join(map(sql.Identifier, metadata_values.keys())),
                    sql.SQL(', ').join(map(sql.Literal, metadata_values.values()))
                )
                query = query.as_string(conn)
                
                with conn.cursor() as cursor:
                    cursor.execute(query)
                conn.commit()
                print(f"  - Inserted {field_config['name']}: {raw_value[:50]}{'...' if len(raw_value) > 50 else ''}")
                
        except Exception as e:
            print(f"Error saving metadata: {e}")
        
    total_time = time.time() - start_time
    print(f"Metadata extraction completed in {total_time:.2f} seconds")
    
    return results

# 2. ENVIRONMENT CONFIGURATION

In [ ]:
# Load environment variables
load_dotenv()

# OpenAI client
openai_key = os.getenv("OPENAI_API_KEY")
client_OPENAI = OpenAI(api_key=openai_key)

# Initialize PostgreSQL connection
conn = psycopg2.connect(
    dbname=os.getenv("DB_NAME"),
    user=os.getenv("DB_USER"),
    host=os.getenv("DB_HOST"),
    port=os.getenv("DB_PORT"),
    password=os.getenv("DB_PASSWORD"),
)
cur = conn.cursor()
conn.autocommit = True

# Register the vector extension
register_vector(conn)

# DATABASE PREPARATION

In [ ]:
# VECTOR EXTENSION
query = "CREATE EXTENSION IF NOT EXISTS vector;"
postgres_execute(conn, query)


## 8.2 Extract Metadata (OpenAI)

In [ ]:
# Example usage for the enhanced metadata extraction function with model selection
document_uuid_to_process = '26ca4b89-7af4-4527-961f-b8db48ec9e95'  # Replace with actual document UUID
#query_ledger_uuid = str(uuid.uuid4())
# Extract metadata using OpenAI
metadata_results = extract_metadata_BEN2(
    document_uuid=document_uuid_to_process,
    conn=conn,
    cur=cur,
    #query_ledger_uuid=query_ledger_uuid,
    client_llm=client_OPENAI,
    llm_method="OPENAI",
    overwrite_existing=True,
    model_name="nomic-embed-text",
    #target_metadata_uuid='650ffea0-2c50-457b-8b24-ab2aeae4a13a',
    username='bfoucque@palo-it.com',
)

print("\nExtracted Metadata Summary:")
for field_name, field_data in metadata_results.items():
    print(f"{field_name}: {field_data}")

In [ ]:
# First, retrieve all document UUIDs from the database
query = "SELECT document_uuid, document_name FROM document_library.documents ORDER BY document_name"
documents = postgres_execute(conn, query, get_result=True)

for i, doc in enumerate(documents):
    document_uuid_to_process = doc['document_uuid']
    #query_ledger_uuid = str(uuid.uuid4())

    # Extract metadata using OpenAI
    metadata_results = extract_metadata_BEN2(
        document_uuid=document_uuid_to_process,
        conn=conn,
        cur=cur,
        #query_ledger_uuid=query_ledger_uuid,
        client_llm=client_OPENAI,
        llm_method="OPENAI",
        overwrite_existing=True,
        model_name="nomic-embed-text",
        #target_metadata_uuid='650ffea0-2c50-457b-8b24-ab2aeae4a13a',
        username='bfoucque@palo-it.com',
    )


## 8.3 Extract Metadata (Ollama)

In [ ]:
# Extract metadata using Ollama with a custom model
ollama_metadata_results = extract_metadata_BEN(
    document_uuid=document_uuid_to_process,
    conn=conn,
    llm_method="OLLAMA",
    overwrite_existing=False,
    model_name="nomic-embed-text"  # Using the enhanced model from pgvector_ben_search
)

print("\nOllama-Extracted Metadata Summary:")
for field_name, field_data in ollama_metadata_results.items():
    print(f"{field_name}: {field_data}")